In [1]:
import numpy as np
from collections import defaultdict

import pickle
import scipy.sparse as sparse
import scipy.sparse.linalg as sparse_linalg
from scipy.spatial import cKDTree

np.set_printoptions(suppress=True)

In [2]:
'''
description：read obj file
input: obj file name
return: obj vertices and face indices
'''
def readOBJ(filename):
    vertices = []
    vertex_norm = []
    vertex_tex = []
    triangles = []
    texcoords = []
    for line in open(filename, "r"):
        values = line.split()
        if(values==[]):
            continue
        if(values=='#'):
            continue
        if(values[0]=='v'):
            vertices.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='vn'):
            vertex_norm.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='vt'):
            vertex_tex.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='f'):
            face=[]
            texcoord = []
            norm = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if(len(w)>=2 and len(w[1])>0):
                    texcoord.append(int(w[1]))
                else:
                    texcoord.append(-1)
                if(len(w)>=3 and len(w[2])>0):
                    norm.append(int(w[2]))
                else:
                    norm.append(-1)
            triangles.append(face)
            texcoords.append(texcoord)
    return np.array(vertices),np.array(triangles)-1

In [3]:
def saveObj(verts,fvidxs,name):
    f = open(name,'w')
    color = np.random.rand(3)
    for v in verts:
        f.write("v {0} {1} {2} {3} {4} {5} \n".format(v[0],v[1],v[2],color[0],color[1],color[2]))
    for fv in fvidxs:
        f.write("f ")
        for ii in range(len(fv)):
            f.write("{0} ".format(fv[ii]+1))
        f.write("\n")
    f.close()

In [4]:
def compute_face_norms(verts,faces):
    fnorms = []
    v4 = []
    # 计算每个面片的三组方向
    for f in faces:
        v1 = verts[f[0]]
        v2 = verts[f[1]]
        v3 = verts[f[2]]
        a = v2 - v1
        b = v3 - v1
        tmp = np.cross(a, b)
        c = tmp.T / np.linalg.norm(tmp)
        fnorms.append([a,b,c])
        # 更新顶点，添加第四个顶点
        v4.append(v1+c)
    fnorms = np.array(fnorms)     
    v4 = np.array(v4)
    new_verts = np.concatenate((verts,v4),axis=0)
    # 更新面片顶点索引
    v4_indices = np.arange(verts.shape[0],verts.shape[0]+v4.shape[0])
    new_faces = np.concatenate((faces,v4_indices.reshape((-1,1))),axis=1)
    return np.transpose(fnorms, (0, 2, 1)),new_verts,new_faces

In [5]:
'''
创建稀疏矩阵
'''
row = np.array([0, 1, 2] * 4)
def expand(f, inv, size):
    i0, i1, i2, i3 = f
    col = np.array([i0, i0, i0, i1, i1, i1, i2, i2, i2, i3, i3, i3])
    data = np.concatenate([-inv.sum(axis=0), *inv])
    return sparse.coo_matrix((data, (row, col)), shape=(3, size), dtype=float)
def construct(faces, invVs, size):
        assert len(faces) == len(invVs)
        return sparse.vstack([expand(f, inv, size) for f, inv in zip(faces, invVs)], dtype=float)

In [6]:
'''
计算邻接三角面
'''
def compute_adjacent_by_edges(objFaces):
    # 每条边涉及到哪些面
    candidates = defaultdict(set)
    for i in range(objFaces.shape[0]):
        f0, f1, f2 = sorted(objFaces[i])
        candidates[(f0, f1)].add(i) # 注意i是面索引
        candidates[(f0, f2)].add(i)
        candidates[(f1, f2)].add(i)
    # 每个面与哪些面邻接；candidates的value存的就是共享边的面
    faces_adjacent = defaultdict(set)  # Face -> Faces
    for faces in candidates.values():
        for f in faces:
            faces_adjacent[f].update(faces)
    # 按面的顺序排列所有邻接面
    faces_sort = []
    for f, adj in faces_adjacent.items():
        exclude_f = []
        for a in adj :
            if a != f:
                exclude_f.append(a)
        faces_sort.append([f,exclude_f])
    faces_sort = sorted(faces_sort, key=lambda e: e[0])
    # 只返回邻接面
    faces_adj = []
    for _,ff in faces_sort:
        faces_adj.append(ff)
    return faces_adj

In [7]:
# 读取原始模型顶点和面片
source_org_v,source_org_f = readOBJ("./model/face/sourceface/face-reference.obj")#readOBJ("./model/cat-lion/cat/cat-reference.obj")#
pose_org_v,pose_org_f = readOBJ("./model/face/sourceface/face-03-fury.obj")#readOBJ("./model/cat-lion/cat/cat-03.obj")#
target_org_v,target_org_f = readOBJ("./model/face/targetface/head-reference.obj")#readOBJ("./model/cat-lion/lion/lion-reference.obj")#

with open("D:/code/python/deformation_transfer/head_map.txt", "rb") as fp:   # Unpickling
    corres = pickle.load(fp)
mapping = np.array(corres)

In [8]:
source_f_norms,source_v,source_f = compute_face_norms(source_org_v,source_org_f)
target_f_norms,target_v,target_f = compute_face_norms(target_org_v,target_org_f)
pose_f_norms,pose_v,pose_f = compute_face_norms(pose_org_v,pose_org_f)

In [9]:
inv_target_span = np.linalg.inv(target_f_norms)
Am = construct(target_f[mapping[:,1]],inv_target_span[mapping[:,1]],len(target_v))
s = (pose_f_norms @ np.linalg.inv(source_f_norms)).transpose(0,2,1)
Bm = np.concatenate(s[mapping[:,0]])

In [10]:
# 没有对应面的部分需要平滑处理
adjacent = compute_adjacent_by_edges(target_f[:,:-1])
inv_target_span = np.linalg.inv(target_f_norms)
missing = np.setdiff1d(np.arange(len(target_f)),np.unique(mapping[:,1]))
count_adjacent = sum(len(adjacent[m]) for m in missing)
if(count_adjacent==0):
    AEs = sparse.csc_matrix((0,len(target_v)),dtype=float)
    Bs = np.zeros((0,3))
else:
    size = len(target_v)
    lhs = []
    rhs = []
    face_idx = 0
    for f,inv in zip(target_f,inv_target_span):
        for adjIndex in adjacent[face_idx]:            
            lhs.append(expand(f,inv,target_v.shape[0]).tocsc())
            rhs.append(expand(target_f[adjIndex],inv_target_span[adjIndex],target_v.shape[0]).tocsc())
        face_idx = face_idx + 1
    AEs = sparse.vstack(lhs) - sparse.vstack(rhs)
    Bs = np.zeros((AEs.shape[0],3))

In [11]:
# 形变迁移
Wm = 1.0 
Ws = 1.0

Astack = [Am*Wm, AEs*Ws]
Bstack = [Bm*Wm, Bs*Ws]
A = sparse.vstack(Astack,format='csc')
A.eliminate_zeros()
b = np.concatenate(Bstack)

In [12]:
LU = sparse.linalg.splu((A.T @ A).tocsc())
x = LU.solve(A.T @ b)
result = x[:source_v.shape[0]]

In [13]:
saveObj(result,target_org_f,"result.obj")